In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [34]:
from nltk.corpus import stopwords

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [37]:
from sklearn.compose import ColumnTransformer

In [38]:
from sklearn.naive_bayes import MultinomialNB

In [39]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV, train_test_split

In [40]:
from sklearn.tree import DecisionTreeClassifier

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
import spacy

In [43]:
vegan = pd.read_csv('vegan.csv', encoding='latin-1')

In [44]:
vegan.head()

,title,selftext,created_utc
0,How do I get involved in Direct Action activism?,NaN,1679864469
1,Is it vegan to buy non-vegan food with 'too go...,'Vegan' friend of mine did. And i honestly was...,1679864006
2,Hmmmm,NaN,1679863672
3,has any damage been done if i havenât been s...,[removed],1679862449
4,Trying to be a vegatarian,[removed],1679861712


In [45]:
vegan['target'] = 0

In [46]:
vegan.head()

,title,selftext,created_utc,target
0,How do I get involved in Direct Action activism?,NaN,1679864469,0
1,Is it vegan to buy non-vegan food with 'too go...,'Vegan' friend of mine did. And i honestly was...,1679864006,0
2,Hmmmm,NaN,1679863672,0
3,has any damage been done if i havenât been s...,[removed],1679862449,0
4,Trying to be a vegatarian,[removed],1679861712,0


In [47]:
plant = pd.read_csv('plant.csv', encoding='latin-1')

In [48]:
plant['target'] = 1

In [49]:
for df in [vegan,plant]:
    df['length'] = df['selftext'].apply(lambda x: len(str(x)))
    df['word_count'] = df['selftext'].apply(lambda x: str(x).count(' '))
    df.dropna(inplace=True)

In [50]:
posts = pd.concat([vegan,plant],axis=0)

In [51]:
posts.head()

,title,selftext,created_utc,target,length,word_count
1,Is it vegan to buy non-vegan food with 'too go...,'Vegan' friend of mine did. And i honestly was...,1679864006,0,550,100
3,has any damage been done if i havenât been s...,[removed],1679862449,0,9,0
4,Trying to be a vegatarian,[removed],1679861712,0,9,0
6,Question for vegans with chickens,"Hi everyone, so Iâm vegan and rescued some c...",1679860400,0,1189,217
7,Crispy Thai chilies with sesame seeds by Chili...,[removed],1679859412,0,9,0


In [52]:
posts.drop_duplicates(subset = 'selftext',keep = 'first',inplace=True)

In [53]:
posts = posts.loc[posts['length'] > 9, :]

In [54]:
posts = posts.loc[posts['word_count'] > 2, :]

In [55]:
posts.head()

,title,selftext,created_utc,target,length,word_count
1,Is it vegan to buy non-vegan food with 'too go...,'Vegan' friend of mine did. And i honestly was...,1679864006,0,550,100
6,Question for vegans with chickens,"Hi everyone, so Iâm vegan and rescued some c...",1679860400,0,1189,217
11,Last friday someone from my college told me it...,So me and a women from my college talked a lit...,1679855229,0,4419,832
15,feeling hopeless about it all,When I'm thinking of all the baby chicks getti...,1679853443,0,763,131
16,The humane slaughter society doesn't seem to k...,The Humane Slaughter Association has released ...,1679853324,0,431,56


In [100]:
import re

In [101]:
nlp = spacy.load('en_core_web_md')

In [119]:
def spacy_tokenizer(post):
    post = str(post).lower()
    post = re.sub('http\S+','', post)
    post = re.sub('[^\w\d\s]','',post)
    post = re.sub('\s+',' ',post)
    doc = nlp(post)
    return " ".join([token.lemma_ for token in doc 
                    if token.pos_.lower() not in ['aux','punct','cconj','det','space','conj','adp','pron','sym']])

In [117]:
from spacy import glossary

In [118]:
for i in dir(spacy.parts_of_speech):
    print(i, glossary.explain(i))

ADJ adjective
ADP adposition
ADV adverb
AUX auxiliary
CCONJ coordinating conjunction
CONJ conjunction
DET determiner
EOL end of line
IDS None
INTJ interjection
NAMES None
NOUN noun
NO_TAG None
NUM numeral
PART particle
PRON pronoun
PROPN proper noun
PUNCT punctuation
SCONJ subordinating conjunction
SPACE space
SYM symbol
VERB verb
X other
__builtins__ None
__doc__ None
__file__ None
__loader__ None
__name__ None
__package__ None
__pyx_unpickle___Pyx_EnumMeta None
__spec__ None
__test__ None
univ_pos_t None


C:\Anaconda\lib\site-packages\spacy\glossary.py:19: UserWarning: [W118] Term 'IDS' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))
C:\Anaconda\lib\site-packages\spacy\glossary.py:19: UserWarning: [W118] Term 'NAMES' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))
C:\Anaconda\lib\site-packages\spacy\glossary.py:19: UserWarning: [W118] Term 'NO_TAG' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))
C:\Anaconda\lib\site-packages\spacy\glossary.py:19: UserWarning: [W118] Term '__builti

In [123]:
posts['selftext'] = [spacy_tokenizer(post) for post in posts['selftext']]

In [23]:
posts.to_csv('posts.csv',index=False)

In [43]:
posts['lemma'] = [token.lemma_ for token in nlp(posts['selftext'])]

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'pandas.core.series.Series'>

In [41]:
[nlp(post) for post in posts['selftext']]


#posts['lemma'] = [nlp(posts['selftext']).lemma_ for lemma in nlp(posts['selftext']) if nlp(posts['selftext']).pos_.lower() 
 #                 not in ['aux','punct','cconj','det','space','conj','adp','pron']]

KeyboardInterrupt: 

In [32]:
words = nlp(posts['selftext'])
[token.lemma_ for token in words if token.pos_.lower() not in ['aux','punct','cconj','det','space','conj','adp','pron']]

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'pandas.core.series.Series'>

In [23]:
nlp = spacy.load('en_core_web_md')
def spacy_tokenizer(string):
    words = nlp(string)
    tokens = [token.lemma_ for token in words if token.pos_.lower() not in ['aux','punct','cconj','det','space','conj','adp','pron']]
    return tokens

In [27]:
spacy_tokenizer(posts['selftext'][0])

['https://www.instagram.com/p/CqQ7ACOvBpe/?igshid=YmMyMTA2M2Y=',
 'talk',
 'use',
 'â\x80\x9cprinciple',
 'â\x80\x9c',
 'bio',
 '-',
 'physics',
 'quantum',
 'mechanicsâ\x80\x9d',
 'to',
 'influence',
 'charge',
 'cell',
 'know',
 'be',
 'new',
 'stuff',
 'go',
 'around',
 'cellular',
 'charge',
 'legitimate',
 'clearly',
 'quackery',
 'more',
 'experience',
 'debate',
 'type',
 'bullshit',
 'help']

In [28]:
[spacy_tokenizer(post) for post in posts['selftext']]

KeyboardInterrupt: 

,index,title,selftext,created_utc,target,length,word_count
0,1,Is it vegan to buy non-vegan food with 'too go...,'Vegan' friend of mine did. And i honestly was...,1679864006,0,550,100
1,6,Question for vegans with chickens,"Hi everyone, so Iâm vegan and rescued some c...",1679860400,0,1190,217
2,11,Last friday someone from my college told me it...,So me and a women from my college talked a lit...,1679855229,0,4455,832
3,15,feeling hopeless about it all,When I'm thinking of all the baby chicks getti...,1679853443,0,769,131
4,16,The humane slaughter society doesn't seem to k...,The Humane Slaughter Association has released ...,1679853324,0,433,56
...,...,...,...,...,...,...,...
24692,21602,If you know of a local event featuring plant-b...,[Living and Eating Green Expo](http://thejourn...,1341375826,1,230,22
24693,21615,HELP! Wanting to start next week!!,My husband and I just watched Forks Over Knive...,1340626413,1,546,107
24694,21616,Various published studies linking weight with ...,[Cardiovascular mortality banded by BMI](http:...,1340594297,1,572,29
24695,21620,Curious about the food rules in the sidebar,Hi! I'm just curious about the following bulle...,1340586971,1,695,133


In [36]:
small = pd.concat([posts.loc[:500,:],posts.loc[20000:20500,:]],axis=0)

In [23]:
X = posts['selftext']
y = posts['target']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify = y, 
                                                    random_state = 42)

In [25]:
X_train.shape

(18522,)

In [26]:
y_train.shape

(18522,)

In [27]:
y_train.value_counts(normalize=True)

0    0.693284
1    0.306716
Name: target, dtype: float64

In [28]:
tk_stop = stopwords.words('english')

In [29]:
mnb_params = {
    'tvec__min_df': [2,3],
    'tvec__max_df': [.9,.95],
    'tvec__ngram_range': [(1,1), (1,2)],
    'tvec__stop_words':['english',None],
    'tvec__max_features':[1000,2000]
}

In [33]:
rf_params = {
    'tvec__min_df': [2,3],
    'tvec__max_df': [.9,.95],
    'tvec__ngram_range': [(1,1), (1,2)],
    'tvec__stop_words':['english',None],
    'tvec__max_features':[1000,2000],
    'rf__n_estimators':[30,50,100]
}

In [30]:
mnb_pipe = Pipeline([
    ('tvec',TfidfVectorizer()),
    ('mnb',MultinomialNB())
])

In [31]:
rf_pipe = Pipeline([
    ('tvec',TfidfVectorizer()),
    ('rf',RandomForestClassifier())
])

In [34]:
gs = GridSearchCV(rf_pipe,
                  param_grid = rf_params,
                  cv=5,
                 verbose=1,
                 n_jobs = -1)

In [35]:
%time 

gs.fit(X_train, y_train)

Wall time: 0 ns
Fitting 5 folds for each of 96 candidates, totalling 480 fits


KeyboardInterrupt: 

In [31]:
gs.best_score_

0.843159964206912

In [152]:
cvec = CountVectorizer()

In [147]:
tfidf = TfidfVectorizer(input = 'content', tokenizer = spacy_tokenizer)

In [155]:
ctx = ColumnTransformer(transformers = [
    ('cvec', CountVectorizer(), ['title','selftext'])
],remainder = 'passthrough')

In [1]:
pipe = Pipeline(steps = [
    ('tfidf', TfidfVectorizer(max_features = 1000)),
    ('mulNB',MultinomialNB())
])

NameError: name 'Pipeline' is not defined

In [177]:
pipe.fit(X_train,y_train)

ValueError: Found input variables with inconsistent numbers of samples: [5, 18587]

In [ ]:
df = []

for token in words:
    tk_dict = {
        'text': token.text,
        'POS': token.pos_,
        'POS_exp':spacy.explain(token.pos_),
        'dependency': token.dep_,
        'dep_exp': spacy.explain(token.dep_),
        'lemma': token.lemma_
    }
    df.append(tk_dict)